$\vec{B}$が既知であるときに、$\vec{B} = \vec{\nabla}{\psi} \times \vec{\nabla}{\chi}$を求める問題を解きたい。

これを式変形すると、
$\vec{B} = \vec{\nabla} \times (\psi\vec{\nabla}{\chi})$
となる。

$\psi_0 = z$

$\chi_0 = xy$

$\vec{B} = \vec{\nabla} \times (\psi\vec{\nabla}{\chi})=(xz,-yz,0)$
という問題をNGSolveで解いてみよう。
$z=0$ の面にはディリクレ条件を課そう。


In [8]:
from ngsolve import *
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw

radius = 0.15
height = 0.50
model = Cylinder((0, 0, 0), (0, 0, 1), radius, height)
model.faces.Min(Z).name = "dir"

geo =OCCGeometry(model)
mesh=Mesh (geo.GenerateMesh(maxh=0.02))

B = CoefficientFunction((x*z, -y*z, 0))
fes_kai = H1(mesh,  order=2, dirichlet="dir", complex=False, symmetry=True)
fes_psi = H1(mesh,  order=2, dirichlet="dir", complex=False, symmetry=True)
kai, N_kai = fes_kai.TnT()
psi, N_psi = fes_psi.TnT()

gfpsi = GridFunction(fes_psi)
gfkai = GridFunction(fes_kai)
gfkai.Set(x*y)

for niter in range(10):
	a = BilinearForm(fes_psi)
	a += InnerProduct(Cross(grad(N_psi),grad(gfkai)),Cross(grad(psi),grad(gfkai)))*dx
	f = LinearForm(fes_psi)
	f += InnerProduct(Cross(grad(N_psi),grad(gfkai)),B)*dx
	a.Assemble()
	f.Assemble()

	gfpsi.vec.data = a.mat.Inverse(fes_psi.FreeDofs()) * f.vec

	a = BilinearForm(fes_kai)
	a += InnerProduct(Cross(grad(gfpsi),grad(N_kai)),Cross(grad(gfpsi),grad(kai)))*dx
	f = LinearForm(fes_kai)
	f += InnerProduct(Cross(grad(gfpsi),grad(N_kai)),B)*dx
	a.Assemble()
	f.Assemble()
	
	gfkai.vec.data = a.mat.Inverse(fes_kai.FreeDofs()) * f.vec


In [9]:
V = Cross(grad(gfpsi),grad(gfkai))
B_ = [[V(mesh(x_,0,z_))[0] for x_ in [-0.1,0.0,0.1]] for z_ in [0.1,0.2,0.3,0.4]]
print(B_)

[[-0.010000232887263831, -3.278896869871988e-06, 0.010009414811134303], [-0.019995804595766213, 7.870073817507975e-06, 0.01999822262407025], [-0.02998239674033462, -7.094054104845245e-06, 0.030015367396593964], [-0.040005190799837, 9.214351749579945e-06, 0.039983285341541955]]


In [11]:
Draw(gfpsi, mesh, clipping = { "pnt" : (0.0,0.0,0), "vec" : (0,1,0) })
Draw(gfkai, mesh, clipping = { "pnt" : (0.0,0.0,0), "vec" : (0,1,0) })

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [12]:
Draw(V, mesh, clipping = { "pnt" : (0.0,0.0,0), "vec" : (0,1,0) })
Draw(B, mesh, clipping = { "pnt" : (0.0,0.0,0), "vec" : (0,1,0) })

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene